# Assignment 9

In [ ]:
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }a
</style>
"""))

#### Installation and import of libraries

In [ ]:
import pandas as pd
#from pandas import Series, DataFrame
import numpy as np
import matplotlib.pyplot as plt 
import chardet

import folium as fm
from folium import Marker, GeoJson
from folium.plugins import MarkerCluster, HeatMap, StripePattern

import geopandas as gpd
from geopandas import GeoSeries
from shapely.geometry import Point, LineString


import branca as br 

### Part 1

#### 1.1 Import the data located at this link. It has information on Tech Institutes' total vacancies, total applicants, total entrants, and total enrolled. Moreover, the institutes are geolocated.

In [ ]:
# Getting the character encoding
# Open the CSV file in binary read mode and read its content as bytes
base = open(r'../../_data/institutos1.csv', 'rb').read()

# Detect the character encoding of the CSV file using the chardet library
det = chardet.detect(base)

# Extract the detected character encoding from the detection result
charenc = det['encoding']
charenc

In [ ]:
# Read the CSV file 'institutos1.csv' into a Pandas DataFrame using the detected character encoding
institutes = pd.read_csv(r'../../_data/institutos1.csv', encoding = charenc)
institutes

In [ ]:
# Select specific columns from the 'institutes' DataFrame and store them in a new DataFrame named 'institutes_data'
institutes_data = institutes[['cod_mod', 'ltimoden_metaatencion', 'cuentadeid_postulante_procesoadm', 'sumaden_flagingresante', 'sumaden_flagmatriculado', 'nlat_ie', 'nlong_ie']]

In [ ]:
# Rename the selected columns in the 'institutes_data' DataFrame to more descriptive names
institutes_data = institutes.rename({'cod_mod':'institute_code','ltimoden_metaatencion':'total_vacancies', 'cuentadeid_postulante_procesoadm':'total_applicants', 'sumaden_flagingresante':'total_entrants', 'sumaden_flagmatriculado':'total_enrolled', 'nlat_ie':'latitude', 'nlong_ie':'longitude'}, axis =1 )
institutes_data

In [ ]:
list(institutes_data.columns)

#### 1.2 Use this shapefile to generate a Choropleth map of the total institutes' vacancies by the department. Use the overlay function (link and JN) to intersect institutes dataset with shapefiles. You can follow this code:

In [ ]:
# Create a GeoDataFrame named 'institutes_geo'
# This GeoDataFrame will contain both the tabular data and geometric data
institutes_geo = gpd.GeoDataFrame(institutes_data,crs="EPSG:4326",
                          geometry=gpd.points_from_xy(institutes_data.longitude, institutes_data.latitude))

In [ ]:
# Read the shapefile containing departmental boundaries into a GeoDataFrame
shapefile_dpto = gpd.read_file(r'../../_data/INEI_LIMITE_DEPARTAMENTAL/INEI_LIMITE_DEPARTAMENTAL.shp')
shapefile_dpto

In [ ]:
# Perform spatial overlay operation to find the intersection of geometries
intersct_inst_geo = gpd.overlay(institutes_geo, shapefile_dpto, how="intersection")

In [ ]:
# Variables: 'total_vacancies', 'total_applicants', 'total_entrants', 'total_enrolled'
vars_sum = ['total_vacancies', 'total_applicants', 'total_entrants','total_enrolled']

In [ ]:
# Group the intersected GeoDataFrame by 'CCDD' (presumably a departmental code)
# and sum the variables in 'vars_sum' for each department
total_dpto = intersct_inst_geo.groupby(['CCDD'], as_index=False)[vars_sum].sum()

# Merge the aggregated data ('total_dpto') back into the departmental shapefile ('shapefile_dpto')
# using 'CCDD' as the common key
total_dpto_shp = shapefile_dpto.merge(total_dpto,on='CCDD')
total_dpto_shp

In [ ]:
list(total_dpto_shp.columns)

In [ ]:
# Specify the latitude and longitude of the Government Palace
lat_palacio = -12.0757538
long_palacio = -76.9863174

# Set the initial zoom level for the map
zoom_start = 5

# Create a Folium Map centered on the Government Palace location
map_graph = fm.Map(location = [lat_palacio, long_palacio], tiles='cartodbpositron', zoom_start = zoom_start)

# Add a choropleth layer to the map
fm.Choropleth(
    geo_data=shapefile_dpto,
    data=total_dpto_shp,
    columns=['CCDD', 'total_vacancies'],
    key_on="feature.properties.CCDD",
    fill_color="YlOrRd",
    fill_opacity=0.8,
    line_opacity=0.2,
    legend_name="TOTAL INSTITUTE VACANCIES BY DEPARTMENT",
    smooth_factor=0,
    Highlight= True,
    line_color = "#0000",
    overlay=True,
    nan_fill_color = "White"  
    ).add_to(map_graph)

# Display the map
map_graph

### Part 2

#### 2.1 Make an interactive map of markes using visual_html function and show total vacancies, applicants, entrants, and enrolled information in a table popup. 

In [ ]:
def visual_html(i):   
    # Extract data for the i-th row from the GeoDataFrame intersct_inst_geo
    Total_Vacancies = intersct_inst_geo['total_vacancies'].iloc[i]                             
    Total_Applicants = intersct_inst_geo['total_applicants'].iloc[i]                           
    Total_Entrants = intersct_inst_geo['total_entrants'].iloc[i]  
    Total_Enrolled = intersct_inst_geo['total_enrolled'].iloc[i]  
    
    # Define colors for left and right columns
    left_col_colour = "#76db37" 
    right_col_colour = "#5ad2d6"
    
    # Create HTML string for the table
    html = """<!DOCTYPE html>
<html>

<head>
    <p> Institutes Data by Department </p>
</head>

<table style="height: 126px; width: 350px;">  <!-- Comment: Create a table. -->
    <tbody> 
"""

    # Define row labels and corresponding data
    row_labels = ["Total Vacancies", "Total Applicants", "Total Entrants", "Total Enrolled"]
    data_values = [Total_Vacancies, Total_Applicants, Total_Entrants, Total_Enrolled]

    # Loop through each row label and data value to generate table rows
    for label, value in zip(row_labels, data_values):
        # Alternate colors for left and right columns
        if label == "Total Vacancies" or label == "Total Entrants":
            left_col_colour = "#76db37"
            right_col_colour = "#5ad2d6"
        else:
            left_col_colour = "#76db37"
            right_col_colour = "#5ad2d6"
        
        # Add row to HTML string
        html += f"""
        <tr>
            <td style="background-color: {left_col_colour};"><span style="color: #1a1719;">{label}</span></td>
            <td style="width: 150px;background-color: {right_col_colour};">{value}</td>
        </tr>
        """

    # Close the HTML table and document tags
    html += """
    </tbody>
</table>
</html>
"""
    return html

#### 2.2 Additionally, use "university" as icon.

In [ ]:
lat_palacio = -12.0757538
long_palacio = -76.9863174
zoom_start = 5

map_graph2 = fm.Map(location = [lat_palacio, long_palacio], tiles='cartodbpositron', zoom_start = zoom_start)

# Iterate over each institute in the intersct_inst_geo GeoDataFrame
for i in range(0,len(intersct_inst_geo)):
    # Generate HTML for the current marker using the visual_html() function
    html = visual_html(i)
    
    # Create an IFrame with the generated HTML
    iframe = br.element.IFrame(html=html,width=350,height=300)
    popup = fm.Popup(iframe, parse_html=True)
    
    # Add a marker to the map at the location of the current institute,
    # with the popup containing detailed information
    fm.Marker([intersct_inst_geo['latitude'].iloc[i],intersct_inst_geo['longitude'].iloc[i]],
                  popup=popup, icon=fm.Icon(color= 'blue', icon='university', prefix="fa")).add_to(map_graph2)

map_graph2